In [3]:
# Import Package
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from IPython.display import display_markdown

In [82]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model = SentenceTransformer("sentence-transformers/paraphrase-MiniLM-L6-v2")


documents = [
    "이차 방정식의 해를 구하는 방법",
    "기하학적 도형의 성질에 관한 설명",
    "삼각형의 각도 계산법",
    "근의 공식은 이차방정식을 푸는 방법 중 하나로, 다음과 같은 식을 사용합니다.",
    "근의 공식은 일반적으로 다음과 같습니다: x = (-b ± √(b^2 - 4ac)) / 2a",
]

document_embeddings = model.encode(documents)

# FAISS 인덱스 생성 및 문서 임베딩 추가
embedding_dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dimension)
index.add(np.array(document_embeddings))


def query_to_embedding(query):

    query_embedding = model.encode([query])
    return query_embedding


def search_similar_documents(query, k=3):

    query_embedding = query_to_embedding(query)

    D, I = index.search(np.array(query_embedding), k=k)

    print("유사한 문서들:")
    for idx in I[0]:
        print(f"- {documents[idx]} (거리: {D[0][idx]})")


# 검색 예시
query = "근의공식에 대해 알려줘"
search_similar_documents(query)

유사한 문서들:
- 이차 방정식의 해를 구하는 방법 (거리: 4.730958461761475)
- 기하학적 도형의 성질에 관한 설명 (거리: 5.489022254943848)
- 삼각형의 각도 계산법 (거리: 5.668970108032227)


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import torch

# 1. 문서 로드 및 텍스트 분리
file_path = "./extended_quadratic_equation_explanation.md"
documents = TextLoader(file_path, encoding="utf8").load()

# 텍스트 분리
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# 2. 임베딩 및 FAISS DB 생성 (KoSimCSE 모델 사용)
embeddings = HuggingFaceEmbeddings()
db = FAISS.from_documents(docs, embeddings)

# 3. 생성 모델 로드 (KoGPT 모델 사용)
gpt_tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")
gpt_model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2")


# 4. RAG 방식으로 답변 생성 함수
def generate_rag_answer(query):
    # FAISS로 검색 수행 (가장 관련 있는 문서 검색)
    search_results = db.similarity_search(query)

    if not search_results:
        return "관련된 문서를 찾을 수 없습니다."

    # 검색된 문서들을 결합하여 문맥을 생성
    context = "\n".join([doc.page_content for doc in search_results])

    # KoGPT 모델을 통해 답변 생성 (검색된 문서를 바탕으로 답변 생성)
    input_text = (
        f"문서에서 검색된 내용을 바탕으로 질문에 답변하세요: {context} 질문: {query}"
    )
    inputs = gpt_tokenizer.encode(input_text, return_tensors="pt")

    # GPU 사용 가능 여부 확인
    device = "cpu"
    gpt_model.to(device)
    inputs = inputs.to(device)

    # 모델을 통해 답변 생성
    outputs = gpt_model.generate(
        inputs,
        max_length=400,
        do_sample=True,
        top_p=0.95,
        top_k=60,
        num_return_sequences=1,
    )

    # 최종 답변 반환
    answer = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


# 5. 예시 질문
query = input("질문을 입력해주세요")

# 6. RAG 방식으로 질문에 대한 답변 생성
answer = generate_rag_answer(query)
display_markdown(answer, raw=True)

Created a chunk of size 335, which is longer than the specified 200
C:\Users\user\AppData\Local\Temp\ipykernel_5892\3310529468.py:17: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


문서에서 검색된 내용을 바탕으로 질문에 답변하세요: 이 방정식의 해는 근의 공식이나 판별식을 통해 쉽게 구할 수 있으며, 방정식의 해는 실수 또는 복소수로 나뉠 수 있습니다. 또한, 이차방정식은 포물선 형태의 그래프와 깊은 관련이 있으며, 물리학, 경제학, 엔지니어링 등 다양한 분야에서 널리 응용됩니다.
## 5. 이차방정식의 응용
이차방정식은 현실 세계의 다양한 문제를 해결하는 데 사용됩니다. 예를 들어:
- **물리학**: 물체의 이동 경로는 종종 이차방정식으로 표현됩니다. 예를 들어, 중력 아래에서 자유 낙하하는 물체의 이동 경로는 포물선 모양을 그리며, 이는 이차방정식으로 설명될 수 있습니다.
- **경제학**: 자본 투자나 수익 최적화 문제에서 이차방정식이 사용될 수 있습니다. 예를 들어, 비용과 수익 간의 관계를 분석하는 데 유용합니다.
- **엔지니어링**: 건축물의 구조적 안정성을 분석할 때 이차방정식이 사용됩니다. 특히 포물선 구조물이나 곡선 형태의 디자인에서 자주 등장합니다.
여기서:
- `a`, `b`, `c`는 상수이며, `a ≠ 0`입니다.
- `x`는 우리가 풀어야 할 미지수입니다.

이차방정식은 그 형태에서 최고 차수가 2인 미지수를 포함하고 있기 때문에 '이차' 방정식이라 불립니다. 이차방정식의 해를 구하기 위해서는 다양한 방법이 사용될 수 있으며, 근의 공식, 인수분해, 또는 그래프를 이용하는 방법 등이 있습니다.
## 4. 이차방정식의 그래프
이차방정식의 그래프는 **포물선**의 형태를 가집니다. 포물선의 방향은 `a`의 부호에 따라 결정됩니다:
- `a > 0`일 때, 그래프는 아래로 볼록한 포물선입니다.
- `a < 0`일 때, 그래프는 위로 볼록한 포물선입니다.

포물선의 꼭짓점은 방정식의 중요한 특징 중 하나이며, 꼭짓점의 x 좌표는 다음과 같이 계산됩니다: 질문: 근의공식에서 [XXX로 정의되는

In [100]:
from transformers import pipeline, ElectraTokenizer, ElectraForSequenceClassification
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# LLM을 위한 전처리 파이프라인 설정
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-discriminator")
model = ElectraForSequenceClassification.from_pretrained(
    "monologg/koelectra-base-discriminator"
)
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# 문서 로드
documents = TextLoader("./이차방정식.txt", encoding="utf8").load()

# 텍스트 분리
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


embeddings = HuggingFaceEmbeddings(model_name="BM-K/KoSimCSE-roberta-multitask")
db = FAISS.from_documents(docs, embeddings)

# 질의를 위한 예시 텍스트
query = "근의 공식"

# LLM을 사용하여 질의를 전처리
processed_query = nlp(query)[0]["label"]

# 처리된 질의로 FAISS 검색
answer = db.similarity_search(processed_query)

# 결과 출력
print(answer)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.


[Document(metadata={'source': './이차방정식.txt'}, page_content='# 이차방정식\n\n## 이차방정식과 그 해\n\n이차방정식은 x에 관한 이차식의 모양으로 정리할 수 있는 등식을 의미한다. \n이차방정식의 일반형은 다음과 같다:\n\n```\nax^2 + bx + c = 0 (단, a ≠ 0)\n```\n\n여기서 a가 0이 되면 이차항이 없어져 이차방정식이 아니므로, a ≠ 0이어야 한다. b나 c는 0이 되어도 상관없다.\n\n### 이차방정식의 해'), Document(metadata={'source': './이차방정식.txt'}, page_content='인수분해는 이차방정식을 푸는 가장 기본적인 방법 중 하나다.\n\n### 예시 1: \n\n```\nx^2 - 7x - 8 = 0\n```\n\n인수분해하여,\n\n```\n(x - 8)(x + 1) = 0\n```\n\n따라서, 해는 x = 8 또는 x = -1이다.\n\n### 예시 2:\n\n```\nx^2 - 8x + 16 = 0\n```\n\n인수분해하면,\n\n```\n(x - 4)^2 = 0\n```\n\n이 경우 중근으로 x = 4가 해가 된다.\n\n### 예시 3:\n\n```\nx^2 + 2x + 8 = 0\n```'), Document(metadata={'source': './이차방정식.txt'}, page_content='이차방정식의 해는 x값을 의미하며, 보통 두 개의 서로 다른 해를 갖는다. \n하지만, 해가 없는 경우 또는 두 해가 중복되는 경우가 있을 수 있다. 중복되는 경우의 해를 중근이라 부른다.\n\n## 이차방정식의 풀이 원리\n\n```\nAB = 0 일 때, A = 0 또는 B = 0\n```\n\n이 원리를 이용하여 이차방정식을 풀이할 수 있다. 예를 들어,\n\n```\n(x - 8)(x + 1) = 0\n```\n\n이 방정식에서 x = 8 또는 x = -1이 해가 된다.\n\n## 인수분해를 이용한 이차방정식의 

In [101]:
display_markdown(answer[0].page_content, raw=True)

# 이차방정식

## 이차방정식과 그 해

이차방정식은 x에 관한 이차식의 모양으로 정리할 수 있는 등식을 의미한다. 
이차방정식의 일반형은 다음과 같다:

```
ax^2 + bx + c = 0 (단, a ≠ 0)
```

여기서 a가 0이 되면 이차항이 없어져 이차방정식이 아니므로, a ≠ 0이어야 한다. b나 c는 0이 되어도 상관없다.

### 이차방정식의 해

In [2]:
embeddings = HuggingFaceBgeEmbeddings()

c:\Users\user\Desktop\Chunjae_edu\.conda\lib\site-packages\IPython\core\interactiveshell.py:3550: LangChainDeprecationWarning: Default values for HuggingFaceBgeEmbeddings.model_name were deprecated in LangChain 0.2.5 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceBgeEmbeddings constructor instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\user\Desktop\Chunjae_edu\.conda\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [73]:
documents = TextLoader("./이차방정식.txt", encoding="utf8").load()

text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

db = FAISS.from_documents(docs, embeddings)

In [93]:
query = "근의 공식"
answer = db.similarity_search(query)

In [94]:
answer

[Document(metadata={'source': './이차방정식.txt'}, page_content='인수분해는 이차방정식을 푸는 가장 기본적인 방법 중 하나다.\n\n### 예시 1: \n\n```\nx^2 - 7x - 8 = 0\n```\n\n인수분해하여,\n\n```\n(x - 8)(x + 1) = 0\n```\n\n따라서, 해는 x = 8 또는 x = -1이다.\n\n### 예시 2:\n\n```\nx^2 - 8x + 16 = 0\n```\n\n인수분해하면,\n\n```\n(x - 4)^2 = 0\n```\n\n이 경우 중근으로 x = 4가 해가 된다.\n\n### 예시 3:\n\n```\nx^2 + 2x + 8 = 0\n```'),
 Document(metadata={'source': './이차방정식.txt'}, page_content='이 방정식은 인수분해가 불가능하다. 이러한 경우는 해가 존재하지 않으며, 허수로 해를 구할 수 있다.\n\n## 이차방정식의 근의 공식\n\n근의 공식은 인수분해가 불가능한 이차방정식을 푸는 데 사용된다.\n\n```\nx = (-b ± √(b^2 - 4ac)) / 2a\n```\n\n여기서 판별식 `D = b^2 - 4ac`를 사용하여 해의 종류를 알 수 있다.\n\n- `D > 0`: 서로 다른 두 실근을 갖는다.\n- `D = 0`: 중근을 갖는다.\n- `D < 0`: 허근을 갖는다.\n\n### 예시:\n\n```\nx^2 + 3x + 2 = 0\n```'),
 Document(metadata={'source': './이차방정식.txt'}, page_content='이차방정식의 해는 x값을 의미하며, 보통 두 개의 서로 다른 해를 갖는다. \n하지만, 해가 없는 경우 또는 두 해가 중복되는 경우가 있을 수 있다. 중복되는 경우의 해를 중근이라 부른다.\n\n## 이차방정식의 풀이 원리\n\n```\nAB = 0 일 때, A = 0 또는 B = 0\n```\n\n이 원리를 이용하여 이차방정식을 풀이할 수 

In [95]:
display_markdown(answer[0].page_content, raw=True)

인수분해는 이차방정식을 푸는 가장 기본적인 방법 중 하나다.

### 예시 1: 

```
x^2 - 7x - 8 = 0
```

인수분해하여,

```
(x - 8)(x + 1) = 0
```

따라서, 해는 x = 8 또는 x = -1이다.

### 예시 2:

```
x^2 - 8x + 16 = 0
```

인수분해하면,

```
(x - 4)^2 = 0
```

이 경우 중근으로 x = 4가 해가 된다.

### 예시 3:

```
x^2 + 2x + 8 = 0
```

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import torch

# 1. 문서 로드 및 텍스트 분리
file_path = "./extended_quadratic_equation_explanation.md"
documents = TextLoader(file_path, encoding="utf8").load()

# 텍스트 분리: 더 큰 문맥 크기를 사용 (chunk_size를 300으로 변경)
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# 2. 임베딩 및 FAISS DB 생성 (KoSimCSE 모델 명확히 설정)
embeddings = HuggingFaceEmbeddings()
db = FAISS.from_documents(docs, embeddings)

# 3. 생성 모델 로드 (KoGPT 모델 사용)
gpt_tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")
gpt_model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2")


# 4. RAG 방식으로 답변 생성 함수
def generate_rag_answer(query):
    # FAISS로 검색 수행 (가장 관련 있는 문서 검색, 검색 결과 수를 3개로 설정)
    search_results = db.similarity_search(query, k=3)

    if not search_results:
        return "관련된 문서를 찾을 수 없습니다."

    # 검색된 문서들을 결합하여 문맥을 생성
    context = "\n".join([doc.page_content for doc in search_results])

    # KoGPT 모델을 통해 답변 생성 (검색된 문서를 바탕으로 답변 생성)
    input_text = (
        f"문서에서 검색된 내용을 바탕으로 질문에 답변하세요: {context} 질문: {query}"
    )
    inputs = gpt_tokenizer.encode(input_text, return_tensors="pt")

    # GPU 사용 가능 여부 확인
    device = "cuda" if torch.cuda.is_available() else "cpu"
    gpt_model.to(device)
    inputs = inputs.to(device)

    # 모델을 통해 답변 생성
    outputs = gpt_model.generate(
        inputs,
        max_length=500,
        do_sample=True,
        top_p=0.95,
        top_k=60,
        num_return_sequences=1,
    )

    # 최종 답변 반환
    answer = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


# 5. 예시 질문
query = input("질문을 입력해주세요: ")

# 6. RAG 방식으로 질문에 대한 답변 생성
answer = generate_rag_answer(query)
print(answer)

Created a chunk of size 335, which is longer than the specified 300
C:\Users\user\AppData\Local\Temp\ipykernel_5892\2335435296.py:17: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


문서에서 검색된 내용을 바탕으로 질문에 답변하세요: 이차방정식은 그 형태에서 최고 차수가 2인 미지수를 포함하고 있기 때문에 '이차' 방정식이라 불립니다. 이차방정식의 해를 구하기 위해서는 다양한 방법이 사용될 수 있으며, 근의 공식, 인수분해, 또는 그래프를 이용하는 방법 등이 있습니다.

## 2. 이차방정식의 해법

### 2.1 근의 공식
근의 공식은 이차방정식을 푸는 가장 일반적인 방법 중 하나입니다. 이 공식은 모든 이차방정식에 적용 가능하며, 특히 인수분해가 어렵거나 불가능할 때 유용합니다. 근의 공식은 다음과 같습니다:
## 5. 이차방정식의 응용
이차방정식은 현실 세계의 다양한 문제를 해결하는 데 사용됩니다. 예를 들어:
- **물리학**: 물체의 이동 경로는 종종 이차방정식으로 표현됩니다. 예를 들어, 중력 아래에서 자유 낙하하는 물체의 이동 경로는 포물선 모양을 그리며, 이는 이차방정식으로 설명될 수 있습니다.
- **경제학**: 자본 투자나 수익 최적화 문제에서 이차방정식이 사용될 수 있습니다. 예를 들어, 비용과 수익 간의 관계를 분석하는 데 유용합니다.
- **엔지니어링**: 건축물의 구조적 안정성을 분석할 때 이차방정식이 사용됩니다. 특히 포물선 구조물이나 곡선 형태의 디자인에서 자주 등장합니다.
따라서 해는 `x = 2`와 `x = -5/2`입니다.

### 연습 문제:
1. `x^2 - 6x + 9 = 0`을 풀어보세요.
2. `2x^2 + 4x - 6 = 0`의 해를 구하세요.
3. `x^2 - x - 2 = 0`을 근의 공식을 사용하여 풀어보세요.

## 4. 이차방정식의 그래프
이차방정식의 그래프는 **포물선**의 형태를 가집니다. 포물선의 방향은 `a`의 부호에 따라 결정됩니다:
- `a > 0`일 때, 그래프는 아래로 볼록한 포물선입니다.
- `a < 0`일 때, 그래프는 위로 볼록한 포물선입니다. 질문: 연습문제를 보여줘야합니다.
1. `a` 또는 `b`<
 - `b><
- `b`<
- `a``<
- `b`<
- `b`